In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import numpy as np
import re
from datetime import datetime, date
import calendar
from calendar import monthrange

In [2]:
driver = webdriver.Chrome()
driver.get("https://gotransam.com/events/2024-Trans-Am-National-Championship/cat/538")

# Wait for dynamic content to load if needed

sched_soup = BeautifulSoup(driver.page_source, 'html.parser')
schedule_details = sched_soup.find_all('div', {'class': 'events-listbox aos-init'})

driver.quit()




In [3]:
# schedule_details

In [4]:
days = [] 
months = []
titles = [] 
descs = []
circuits = [] 
year = str(datetime.now().year)

tit_prt = 'Trans-Am Race '
rnd_cnt = 1
titles = []

month_mapping = {
        'Jan': 1,
        'JAN': 1,
        'January': 1,
        'Feb': 2,
        'FEB': 2,
        'Mar': 3,
        'March': 3,
        'Apr': 4,
        'May': 5,
        'Jun': 6,
        'Jul': 7,
        'Aug': 8,
        'Sep': 9,
        'Oct': 10,
        'October': 10,
        'Nov': 11,
        'Dec': 12,
        'DEC': 12
    }



In [5]:
len(schedule_details)

12

In [6]:
f = schedule_details[0].find('div', {'class':"event-calendar"}).text.split()#[0].split('-')
f

['Feb', '22-25']

In [7]:
dates = []

for i in schedule_details:
    day_ints = []
    scrp_daystr = []
    dates = []
    wknd_dates = []
    
    temp_date = i.find('div', {'class':"event-calendar"}).text.split()    
    temp_month = temp_date[0]
    
    #   Logic for multi-month events
    if '-' in temp_month:
        temp_start_month = temp_month.split('-')[0]
        temp_end_month = temp_month.split('-')[1]
        converted_start_month = month_mapping[temp_start_month]
        converted_end_month = month_mapping[temp_end_month]
#         print('converted_start_month: ' + str(converted_start_month), 'converted_end_month: ' + str(converted_end_month))

        scrp_day = temp_date[1].split('-')
        start_day, end_day = map(int, scrp_day)  # start_day = 31, end_day = 3
#         print('start_day: ' + str(start_day), 'end_day: ' + str(end_day))
        
        start_date = year + '-' + str(converted_start_month).zfill(2) + '-' + str(start_day).zfill(2)      
#         print('start_date: ' + str(start_date))

        end_date = year + '-' + str(converted_end_month).zfill(2) + '-' + str(end_day).zfill(2)
#         print('end_date: ' + str(end_date))

        _, last_day_of_month = monthrange(int(year), converted_start_month)
        days_in_strt_month = np.arange(start_day, last_day_of_month + 1).tolist()
#         print('last_day_of_month: ' + str(last_day_of_month), 'days_in_strt_month: ' + str(days_in_strt_month) ) 
        
        days_in_end_month = np.arange(1, end_day + 1).tolist()
#         print('days_in_end_month: ' + str(days_in_end_month) ) 
            
        for day in days_in_strt_month:
            date_str = year + '-' + str(converted_start_month).zfill(2) + '-' + str(day).zfill(2)
            dates.append(date_str)
            
        for day in days_in_end_month:
            date_str = year + '-' + str(converted_end_month).zfill(2) + '-' + str(day).zfill(2)
            dates.append(date_str)
            
    else:
#         logic for single month events
        converted_month = month_mapping[temp_month]
#         print('converted_month: ' + str(converted_month))
        
        scrp_day = temp_date[1].split('-')
        start_day, end_day = map(int, scrp_day)  # start_day = 22, end_day = 25
#         print('start_day: ' + str(start_day), 'end_day: ' + str(end_day))
        
        start_date = year + '-' + str(converted_month).zfill(2) + '-' + str(start_day).zfill(2)      
#         print('start_date: ' + str(start_date))

        end_date = year + '-' + str(converted_month).zfill(2) + '-' + str(end_day).zfill(2)
#         print('end_date: ' + str(end_date))
        
        for day in range(start_day, end_day + 1):
            date_str = year + '-' + str(converted_month).zfill(2) + '-' + str(day).zfill(2)
            dates.append(date_str)
        
#     print('Dates:', ', '.join(dates))
    
    
    for date in dates:
        date_obj = datetime.strptime(date, '%Y-%m-%d')

        year, month, day = date_obj.year, date_obj.month, date_obj.day
        if calendar.weekday(year, month, day) >= 5:
            wknd_dates.append(date)
#     print('wknd_dates: ' + ', '.join(wknd_dates))
    
    for date in wknd_dates:
        year, month, day = date.split('-')
        day = int(day) # Remove leading zero if present
        month = int(month) # Remove leading zero if present
        days.append(day)
        months.append(month)

        
    
    if len(wknd_dates) == 2:   
        temp_circuit = i.find('div', {'class':'event-destination'}).text
        temp_circuit = re.split('Tour\s*', temp_circuit)[-1]  # Split on 'Tour' followed by any number of spaces
        circuits.extend([temp_circuit] * 2)

        tit = tit_prt + str(rnd_cnt)
        rnd_cnt += 1
        titles.extend([tit] * 2) 
        
        desc_1 = 'TA2 Race'
        descs.append(desc_1)
        desc_2 = 'TA1 and Friends Race'
        descs.append(desc_2)
        
    elif len(wknd_dates) == 1:
        temp_circuit = i.find('div', {'class':'event-destination'}).text
        temp_circuit = re.split('Tour\s*', temp_circuit)[-1]  # Split on 'Tour' followed by any number of spaces
        circuits.append(temp_circuit)

        tit = tit_prt + str(rnd_cnt)
        rnd_cnt += 1
        titles.append(tit) 

        desc_1 = 'TA2 Race'
        descs.append(desc_1)


    
    

#### If you need to handle the case where the range spans over a year, you can add logic to update the year accordingly.

# # Checking if the range spans over a year
# if converted_start_month > converted_end_month:
#     next_year = str(int(year) + 1)
#     for month in range(1, converted_end_month + 1):
#         start_date = next_year + '-' + str(month).zfill(2) + '-' + str(start_day).zfill(2)
#         _, last_day_of_month = monthrange(int(next_year), month)
#         days_in_month = np.arange(start_day, last_day_of_month + 1).tolist()
        
#         for day in days_in_month:
#             date_str = next_year + '-' + str(month).zfill(2) + '-' + str(day).zfill(2)
#             dates.append(date_str)
    
#     print('Dates:', ', '.join(dates))

In [8]:
titles

['Trans-Am Race 1',
 'Trans-Am Race 1',
 'Trans-Am Race 2',
 'Trans-Am Race 2',
 'Trans-Am Race 3',
 'Trans-Am Race 3',
 'Trans-Am Race 4',
 'Trans-Am Race 5',
 'Trans-Am Race 5',
 'Trans-Am Race 6',
 'Trans-Am Race 6',
 'Trans-Am Race 7',
 'Trans-Am Race 7',
 'Trans-Am Race 8',
 'Trans-Am Race 8',
 'Trans-Am Race 9',
 'Trans-Am Race 9',
 'Trans-Am Race 10',
 'Trans-Am Race 10',
 'Trans-Am Race 11',
 'Trans-Am Race 11',
 'Trans-Am Race 12',
 'Trans-Am Race 12']

In [9]:
circuits

['Sebring International Raceway\nSebring, FL\n',
 'Sebring International Raceway\nSebring, FL\n',
 'Road Atlanta \nBraselton, GA\n',
 'Road Atlanta \nBraselton, GA\n',
 'NOLA Motorsports Park\nAvondale, LA\n',
 'NOLA Motorsports Park\nAvondale, LA\n',
 'World Wide Technology Raceway  (TA2 Only)\nMadison, IL\n',
 'Lime Rock Park\nLakeville, CT\n',
 'Lime Rock Park\nLakeville, CT\n',
 'Pittsburgh International Race Complex\nWampum, PA\n',
 'Pittsburgh International Race Complex\nWampum, PA\n',
 'Mid-Ohio Sports Car Course\nLexington, OH\n',
 'Mid-Ohio Sports Car Course\nLexington, OH\n',
 'Road America\nElkhart Lake, WI\n',
 'Road America\nElkhart Lake, WI\n',
 'Watkins Glen International\nWatkins Glen, NY\n',
 'Watkins Glen International\nWatkins Glen, NY\n',
 'Canadian Tire Motorsport Park\nBowmanville, ON\n',
 'Canadian Tire Motorsport Park\nBowmanville, ON\n',
 'VIRginia International Raceway\nAlton, VA\n',
 'VIRginia International Raceway\nAlton, VA\n',
 'Circuit of The Americas\nAu

In [10]:
descs

['TA2 Race',
 'TA1 and Friends Race',
 'TA2 Race',
 'TA1 and Friends Race',
 'TA2 Race',
 'TA1 and Friends Race',
 'TA2 Race',
 'TA2 Race',
 'TA1 and Friends Race',
 'TA2 Race',
 'TA1 and Friends Race',
 'TA2 Race',
 'TA1 and Friends Race',
 'TA2 Race',
 'TA1 and Friends Race',
 'TA2 Race',
 'TA1 and Friends Race',
 'TA2 Race',
 'TA1 and Friends Race',
 'TA2 Race',
 'TA1 and Friends Race',
 'TA2 Race',
 'TA1 and Friends Race']

In [11]:
months

[2, 2, 3, 3, 4, 4, 5, 5, 5, 6, 6, 6, 6, 6, 6, 8, 8, 8, 9, 10, 10, 11, 11]

In [12]:
days

[24,
 25,
 23,
 24,
 13,
 14,
 11,
 25,
 26,
 8,
 9,
 22,
 23,
 29,
 30,
 24,
 25,
 31,
 1,
 5,
 6,
 2,
 3]

In [13]:
year

'2024'

In [14]:
import pytz
import icalendar
from datetime import datetime
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [15]:
Trans_Am = Calendar()
Trans_Am.add('prodid', '-//My calendar product//example.com//')
Trans_Am.add('version', '2.0')
uid = 110

In [18]:
 
for title, circuit, day, month, desc in zip(titles, circuits, days, months, descs):
    ievent = Event()
    ievent.add('summary', title) #Title of the event
    ievent.add('description', desc)     #Description of event
    #     ievent.add('description', f"{event}\n{length}")

    ievent.add('dtstart', datetime(int(year), month, day, 14, 0, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtend', datetime(int(year), month, day, 16, 0, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtstamp', datetime(2024, 1, 16, 0, 10, 0, tzinfo=pytz.timezone('est')))
    ievent.add('location', circuit)
    ievent.add('uid', uid)
    uid += 1
    Trans_Am.add_component(ievent)

In [19]:
Trans_Am

VCALENDAR({'PRODID': vText('b'-//My calendar product//example.com//''), 'VERSION': vText('b'2.0'')}, VEVENT({'SUMMARY': vText('b'Trans-Am Race 1''), 'DESCRIPTION': vText('b'TA2 Race''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x000001F46892C4C0>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x000001F4685A7FA0>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x000001F4685A7D60>, 'LOCATION': vText('b'Sebring International Raceway\\nSebring\\, FL\\n''), 'UID': vText('b'110'')}), VEVENT({'SUMMARY': vText('b'Trans-Am Race 1''), 'DESCRIPTION': vText('b'TA1 and Friends Race''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x000001F4685A7580>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x000001F4685A74C0>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x000001F4685A7880>, 'LOCATION': vText('b'Sebring International Raceway\\nSebring\\, FL\\n''), 'UID': vText('b'111'')}), VEVENT({'SUMMARY': vText('b'Trans-Am Race 2''), 'DESCRIPTION': vText('b'TA2 Race''), 'DTSTART': <icalendar.prop.v

In [20]:
directory = str(Path(r'C:\Users\chris\Documents\Calendars')) + "/"
print("ics file will be generated at ", directory)
f = open(os.path.join(directory, 'Trans-Am.ics'), 'wb')
f.write(Trans_Am.to_ical())
f.close()

ics file will be generated at  C:\Users\chris\Documents\Calendars/
